In [1]:
print('Inicializando Variáveis do Programa')
#inicializando variáveis
#lista_confianças = [0.01]
lista_confianças = [0.01, 0.05]
variável_dependente = 'qty_2019'
variáveis_independentes = ('qty_2014', 'C(grupo)', '-1')
ano_início = 2014
ano_fim = 2020
pareamento=False
capes=True
cnpq=True
nenhum=True
ambos=True
verbose = False

Inicializando Variáveis do Programa


In [2]:
from datetime import datetime
print('Início em: ', datetime.now())

print('Importações Iniciais')

from tkinter import NONE
from Lattes import Lattes
from Carga import Carga
from Database import Database
import psycopg2, pickle
import pandas, os
import statsmodels.api as sm
from urllib.parse import urlencode
from psycopg2.extensions import AsIs
from Carga import Carga
from pandas import pandas as pd
import numpy as np
import Database
import json
import locale
import statsmodels.formula.api as sm2
from itertools import combinations




print('Fim: ', datetime.now())

Início em:  2023-04-23 23:29:21.535405
Importações Iniciais
Fim:  2023-04-23 23:29:24.325592


In [3]:
sql = '''
select * from financiamentos_doutorandos_2014
    inner join indicadores_doutorado
        on indicadores_doutorado.id = financiamentos_doutorandos_2014.id
    inner join indicadores_nomes
        on indicadores_nomes.tipo = indicadores_doutorado.indicador_tipo                 
'''

In [4]:
#Carregar arquivo na memória
print('Carregando indicadores na memória. Início em: ', datetime.now())

db = Database.Database('CNPq')
engine = Carga.db_engine()

dt = pd.read_sql(sql, engine)
if verbose: print(f'dt.size: {dt.size}')

print('Término em: ', datetime.now())

Carregando indicadores na memória. Início em:  2023-04-23 23:29:24.386353
Término em:  2023-04-23 23:29:34.155802


In [5]:
sql_ids_pareados = '''
select pareado_1 as id from public.par_doutorandos_2014_sem_reposicao
UNION select pareado_2 as id from public.par_doutorandos_2014_sem_reposicao
UNION select pareado_3 as id from public.par_doutorandos_2014_sem_reposicao
'''

In [6]:
print('Carregando indicadores dos pareados na memória. Início em: ', datetime.now())

db = Database.Database('CNPq')
engine = Carga.db_engine()

pareados = pd.read_sql(sql_ids_pareados, engine)
if verbose: print(f'dt.size: {dt.size}')

print('Término em: ', datetime.now())

Carregando indicadores dos pareados na memória. Início em:  2023-04-23 23:29:34.206615
Término em:  2023-04-23 23:29:34.345369


In [7]:
## Normalizando as Tabelas

print('Realizando normalização da tabela. Início em: ', datetime.now())

dt['const'] = 1
dt = dt.loc[:,~dt.columns.duplicated()]

dt['bool_cnpq'] = dt['pagtos'].map(lambda pagtos: 1 if pagtos > 0 else 0)
dt['bool_capes'] = dt['capes'].map(lambda capes: 1 if capes == True else 0)



#Preenchendo Nan com zeros nos indicadores
dt.qty_2012 = dt.qty_2012.fillna(0)
dt.qty_2013 = dt.qty_2013.fillna(0)
dt.qty_2014 = dt.qty_2014.fillna(0)
dt.qty_2015 = dt.qty_2015.fillna(0)
dt.qty_2016 = dt.qty_2016.fillna(0)
dt.qty_2017 = dt.qty_2017.fillna(0)
dt.qty_2018 = dt.qty_2018.fillna(0)
dt.qty_2019 = dt.qty_2019.fillna(0)
dt.capes = dt.capes.fillna(0)
dt.pagtos = dt.pagtos.fillna(0)

valor_total_cnpq =  dt[['id', 'pagtos']].loc[dt['pagtos']>0].drop_duplicates().sum().pagtos
dt['porcent_pagos'] = dt['pagtos']/valor_total_cnpq


print('Fim em: ', datetime.now())


Realizando normalização da tabela. Início em:  2023-04-23 23:29:34.367718
Fim em:  2023-04-23 23:29:35.328266


In [8]:
dt['grupo'] = dt.apply(lambda d: 'Ambos' if (d['capes'] == True and d['pagtos']>0) \
                       else 'CAPES' if (d['capes'] == True and d['pagtos']==0) \
                       else 'CNPq' if d['pagtos'] > 0 \
                       else 'Nenhum', axis=1)


In [9]:
#Apagando os financiados 
if not capes:
    dt.drop(dt.loc[dt.grupo == 'CAPES'].index, inplace=True)
if not cnpq:
    dt.drop(dt.loc[dt.grupo == 'CNPq'].index, inplace=True)
if not nenhum:
    dt.drop(dt.loc[dt.grupo == 'Nenhum'].index, inplace=True)
if not ambos:
    dt.drop(dt.loc[dt.grupo == 'Ambos'].index, inplace=True)

#Apagando os que não foram escolhidos pelo pareamento
if pareamento:   
    dt.drop(dt.loc[~dt.id.isin(pareados.id)].loc[dt.pagtos==0].index, inplace=True)

In [10]:
def regressão(X,y):
    try:
        # OLS vem de Ordinary Least Squares e o método fit irá treinar o modelo
        reg = sm.OLS(y, X).fit()
        # mostrando as estatísticas do modelo
        #reg.summary()
        
        
        
        #guardando a regressão na memória

        result = {
                'Indicador': indicador,
                'Erro': False,
                'Prob (F-statistic)':reg.f_pvalue, 
                'Parâmetros': reg.params, 
                "P>|t|":reg.pvalues, 
                'Standard Error': reg.bse, 
                "Covariância entre os pasrâmetros": reg.normalized_cov_params, 
                'Number of observations n.': reg.nobs, 
                "R2 ajustado": reg.rsquared_adj, 
                "Modelo":reg.model,
            }
    except:
        result = {
                'Indicador': indicador,
                'Erro': True,
                'Prob (F-statistic)':None, 
                'Parâmetros': None, 
                "P>|t|":None, 
                'Standard Error': None, 
                "Covariância entre os pasrâmetros": None, 
                'Number of observations n.': None, 
                "R2 ajustado": None, 
                "Modelo": None,
            }
    return result

In [11]:
def regressão_r2(dados, variável_dependente, variáveis_independentes):
    
    lista_formulas = []
    
    for x in range(1,len(variáveis_independentes)+1):
        for a in combinations(variáveis_independentes,x):
            formula = f'{variável_dependente} ~ ' 
            if not a[0] == '-1':
                formula = (formula + ' + '.join(a)).replace('+ -1',' -1')
                lista_formulas.append(formula)
    
    list_results = []
    for formula in lista_formulas:
    
        try:
            # OLS vem de Ordinary Least Squares e o método fit irá treinar o modelo
            reg_ajustado = sm2.ols(formula, data = dados)
            reg = reg_ajustado.fit()
            # mostrando as estatísticas do modelo
            if verbose: print('\nCom constante:\n',reg.summary2())  

            #guardando a regressão na memória

            result = {
                    'Indicador': indicador,
                    'Erro': False,
                    'Prob (F-statistic)':reg.f_pvalue, 
                    'Parâmetros': reg.params, 
                    "P>|t|":reg.pvalues, 
                    'Standard Error': reg.bse, 
                    "Covariância entre os pasrâmetros": reg.normalized_cov_params, 
                    'Number of observations n.': reg.nobs, 
                    "R2 ajustado": reg.rsquared_adj, 
                    "Modelo":reg.model,
                }

        except:
            result = {
                    'Indicador': indicador,
                    'Erro': True,
                    'Prob (F-statistic)':None, 
                    'Parâmetros': None, 
                    "P>|t|":None, 
                    'Standard Error': None, 
                    "Covariância entre os pasrâmetros": None, 
                    'Number of observations n.': None, 
                    "R2 ajustado": None, 
                    "Modelo": None,
                }
    
        list_results.append(result)
        
    maiorR = max(list_results, key=lambda x:x['R2 ajustado'])
        
    return maiorR

In [12]:
def regressão_com_confiança (X,y,confiança = 0.05): 
    result = regressão(X,y)
    if verbose: print(result)
    while result['Erro'] == False and result['P>|t|'].max() > confiança:
        X = X.drop(columns=result['P>|t|'].idxmax())
        result = regressão(X,y)
        if verbose: print(result)
        
        
    if result['Erro'] == False and result['Prob (F-statistic)'] < confiança and result['P>|t|'].max() < confiança:
        result['Confiança'] = confiança
        return result
    else:
        result['Confiança'] = 'Erro'
    return result


In [13]:
#inicializando variáveis
lista_regressões = []
tempo_início = datetime.now()
feitos = 0
print(f'Iniciado em {tempo_início}.')

#pegando lista de todos os indicadores
todos_indicadores = dt.nome.unique()

#iterando sobre cada indicador
número_total = len(todos_indicadores)
for indicador in todos_indicadores:
    
    #Realizando previsão de término
    if not feitos == 0:
        porcentagem_já_feita = (feitos/número_total)
        tempo_passado = datetime.now() - tempo_início
        tempo_por_id = tempo_passado / feitos
        tempo_restante = (número_total - feitos) * tempo_por_id
        tempo_em_que_vai_acabar = datetime.now() + tempo_restante
        print(f'{feitos}/{número_total}. {porcentagem_já_feita * 100}% feitos. Fazendo indicador: {indicador}. Acabará em {tempo_em_que_vai_acabar}. O Último demorou: {datetime.now() - tempo_último}.')
    else:
        print(f'{feitos}/{número_total}. Fazendo id: {indicador}.')
    feitos += 1
    tempo_último = datetime.now()

    #Realizando a regressão
    dados = dt.loc[dt.nome==indicador]
    #X = dados[['const','qty_2014', 'bool_cnpq']]
    #y = dados[['qty_2019']]
    #lista_confianças = (0.001, 0.01, 0.05, 0.1)
    result = regressão_r2(dados, variável_dependente, variáveis_independentes)


  

    lista_regressões.append(result)


Iniciado em 2023-04-23 23:29:47.345093.
0/130. Fazendo id: Prêmio ou Título Recebido.
1/130. 0.7692307692307693% feitos. Fazendo indicador: Conclusão de Doutorado. Acabará em 2023-04-23 23:30:57.295623. O Último demorou: 0:00:00.370421.
2/130. 1.5384615384615385% feitos. Fazendo indicador: Conclusão de Especialização. Acabará em 2023-04-23 23:30:53.255092. O Último demorou: 0:00:00.475918.
3/130. 2.307692307692308% feitos. Fazendo indicador: Conclusão de Graduação. Acabará em 2023-04-23 23:30:43.598044. O Último demorou: 0:00:00.283148.
4/130. 3.076923076923077% feitos. Fazendo indicador: Conclusão de Mestrado. Acabará em 2023-04-23 23:30:44.463975. O Último demorou: 0:00:00.459360.
5/130. 3.8461538461538463% feitos. Fazendo indicador: Início de Doutorado. Acabará em 2023-04-23 23:30:45.077441. O Último demorou: 0:00:00.462967.
6/130. 4.615384615384616% feitos. Fazendo indicador: Início de Especialização. Acabará em 2023-04-23 23:30:45.716785. O Último demorou: 0:00:00.473607.
7/130. 5

54/130. 41.53846153846154% feitos. Fazendo indicador: Desenvolvimento de Material Didático ou Instrucional. Acabará em 2023-04-23 23:30:33.265620. O Último demorou: 0:00:00.268338.
55/130. 42.30769230769231% feitos. Fazendo indicador: Outra produção Técnica. Acabará em 2023-04-23 23:30:32.917905. O Último demorou: 0:00:00.206145.
56/130. 43.07692307692308% feitos. Fazendo indicador: Outras Participações de Bancas Julgadoras. Acabará em 2023-04-23 23:30:32.743446. O Último demorou: 0:00:00.275409.
57/130. 43.84615384615385% feitos. Fazendo indicador: Participação em Exposição. Acabará em 2023-04-23 23:30:32.588723. O Último demorou: 0:00:00.281365.
58/130. 44.61538461538462% feitos. Fazendo indicador: Publicação de Prefácio ou Posfácio. Acabará em 2023-04-23 23:30:32.143718. O Último demorou: 0:00:00.149479.
59/130. 45.38461538461539% feitos. Fazendo indicador: Programa de Rádio ou TV. Acabará em 2023-04-23 23:30:31.681225. O Último demorou: 0:00:00.134730.
60/130. 46.15384615384615% fe

106/130. 81.53846153846153% feitos. Fazendo indicador: Produção Técnica - Produto Tecnológico. Acabará em 2023-04-23 23:30:22.328887. O Último demorou: 0:00:00.145043.
107/130. 82.3076923076923% feitos. Fazendo indicador: Orientação em Andamento de Aperfeiçoamento ou Especialização. Acabará em 2023-04-23 23:30:22.173307. O Último demorou: 0:00:00.141057.
108/130. 83.07692307692308% feitos. Fazendo indicador: Patente Registrada. Acabará em 2023-04-23 23:30:22.066091. O Último demorou: 0:00:00.178821.
109/130. 83.84615384615385% feitos. Fazendo indicador: Participação em Banca Julgadora para Professor Titular. Acabará em 2023-04-23 23:30:21.999406. O Último demorou: 0:00:00.211173.
110/130. 84.61538461538461% feitos. Fazendo indicador: Desenvolvimento de Desenho Industrial. Acabará em 2023-04-23 23:30:21.862660. O Último demorou: 0:00:00.150866.
111/130. 85.38461538461539% feitos. Fazendo indicador: Conclusão de Residência Médica. Acabará em 2023-04-23 23:30:21.704329. O Último demorou: 

C:\Python\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1804: RuntimeWarning: divide by zero encountered in double_scalars
  return self.mse_model/self.mse_resid


119/130. 91.53846153846153% feitos. Fazendo indicador: Orientação em Andamento de Pós-Doutorado. Acabará em 2023-04-23 23:30:20.559665. O Último demorou: 0:00:00.128405.
120/130. 92.3076923076923% feitos. Fazendo indicador: Participação em Banca Julgadora para Livre Docência. Acabará em 2023-04-23 23:30:20.413798. O Último demorou: 0:00:00.120850.
121/130. 93.07692307692308% feitos. Fazendo indicador: Manutenção de Obra Artística. Acabará em 2023-04-23 23:30:20.295423. O Último demorou: 0:00:00.144199.
122/130. 93.84615384615384% feitos. Fazendo indicador: Solicitação de Cultivar protegida. Acabará em 2023-04-23 23:30:20.183383. O Último demorou: 0:00:00.148320.
123/130. 94.61538461538461% feitos. Fazendo indicador: Maquete. Acabará em 2023-04-23 23:30:20.069898. O Último demorou: 0:00:00.144227.
124/130. 95.38461538461539% feitos. Fazendo indicador: Publicação de Partitura Musical. Acabará em 2023-04-23 23:30:19.970888. O Último demorou: 0:00:00.157285.
125/130. 96.15384615384616% fei

C:\Python\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1728: RuntimeWarning: invalid value encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Python\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Python\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1728: RuntimeWarning: invalid value encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Python\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Python\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1728: RuntimeWarning: invalid value encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
C:\Python

In [14]:
regressões = pd.DataFrame(lista_regressões)
regressões = pd.concat([regressões, 
                        regressões['Parâmetros'].apply(pd.Series), 
                        regressões['P>|t|'].apply(pd.Series), 
                        regressões['Standard Error'].apply(pd.Series)], 
                       axis=1)
regressões.to_excel(f'Regressões dos Doutorandos {ano_início} com Pareamento {pareamento} e sem Reposição CNPq {cnpq} CAPES {capes} Ambos {ambos} Nenhum {nenhum}.xlsx')